In [3]:
import csv
import datetime
import os
import yaml
from itertools import islice

from nupic.frameworks.opf.model_factory import ModelFactory

_NUM_RECORDS = 3000
_INPUT_FILE_PATH = "C:\Users\mossa\Nupic_doc\gymdata.csv"
_PARAMS_PATH = "C:\Users\mossa\Nupic_doc\model.yaml"

#Nupicのmodelfactoryとモデルの中身を記述したyamlファイルを使ってOPFモデルを作成する

def createModel():
    with open(_PARAMS_PATH, "r") as f:
        modelParams = yaml.safe_load(f)
    return ModelFactory.create(modelParams)



def runHotgym(numRecords):
    model = createModel()
    #予測のデータを返却する
    model.enableInference({"predictedField": "consumption"})
    with open(_INPUT_FILE_PATH) as fin:
        reader = csv.reader(fin)
        headers = reader.next()
        reader.next()
        reader.next()
        results = []
        for record in islice(reader, numRecords):
            modelInput = dict(zip(headers, record))
            modelInput["consumption"] = float(modelInput["consumption"])
            modelInput["timestamp"] = datetime.datetime.strptime(modelInput["timestamp"], "%m/%d/%y %H:%M")
            result = model.run(modelInput)
            bestPredictions = result.inferences["multiStepBestPredictions"]
            allPredictions = result.inferences["multiStepPredictions"]
            oneStep = bestPredictions[1]
            oneStepConfidence = allPredictions[1][oneStep]
            fiveStep = bestPredictions[5]
            fiveStepConfidence = allPredictions[5][fiveStep]
        
            result = (oneStep, oneStepConfidence * 100,fiveStep, fiveStepConfidence * 100)
            print "1-step: {:16} ({:4.4}%)\t 5-step: {:16} ({:4.4}%)".format(*result)
            results.append(result)
    return results


if __name__ == "__main__":
    runHotgym(_NUM_RECORDS)

1-step:             21.2 (100.0%)	 5-step:             21.2 (100.0%)
1-step:             16.4 (99.8%)	 5-step:             16.4 (99.8%)
1-step:              4.7 (99.6%)	 5-step:              4.7 (99.6%)
1-step:              4.7 (99.6%)	 5-step:              4.7 (99.6%)
1-step:              4.6 (99.4%)	 5-step:              4.6 (99.4%)
1-step:             23.5 (99.4%)	 5-step:             23.5 (99.4%)
1-step:             47.5 (99.21%)	 5-step:             47.5 (99.21%)
1-step:             45.4 (99.06%)	 5-step:             45.4 (99.06%)
1-step:             46.1 (98.87%)	 5-step:             46.1 (98.87%)
1-step:             41.5 (98.87%)	 5-step:             41.5 (98.87%)
1-step:             43.4 (98.68%)	 5-step:             43.4 (98.68%)
1-step:            45.61 (96.41%)	 5-step:             43.8 (94.31%)
1-step:             37.8 (83.42%)	 5-step:             37.8 (90.28%)
1-step:             36.6 (86.53%)	 5-step:             36.6 (90.13%)
1-step:             4.67 (79.87%)	 5-step:  

1-step:    4.65850461565 (100.0%)	 5-step:    38.9483378705 (100.0%)
1-step:    4.65850461565 (100.0%)	 5-step:         38.05986 (100.0%)
1-step:       22.2672168 (100.0%)	 5-step:          40.6145 (100.0%)
1-step:           42.388 (100.0%)	 5-step:          44.0271 (100.0%)
1-step:    38.9483378705 (99.81%)	 5-step:           41.443 (100.0%)
1-step:       43.3811162 (99.93%)	 5-step:          40.6145 (100.0%)
1-step:          40.6145 (100.0%)	 5-step:        39.882363 (100.0%)
1-step:      43.23678134 (100.0%)	 5-step:    38.9483378705 (100.0%)
1-step:           41.443 (100.0%)	 5-step:    38.9483378705 (100.0%)
1-step:          40.6145 (100.0%)	 5-step:      36.98072232 (100.0%)
1-step:       39.7376541 (100.0%)	 5-step:          40.6145 (100.0%)
1-step:    38.9483378705 (100.0%)	 5-step:      39.66635787 (100.0%)
1-step:          44.0271 (99.99%)	 5-step:      43.23678134 (100.0%)
1-step:     37.046505624 (100.0%)	 5-step:     37.046505624 (100.0%)
1-step:          40.6145 (100.0%)	

1-step:             39.4 (86.15%)	 5-step:             39.4 (88.54%)
1-step:             41.3 (84.45%)	 5-step:             41.3 (86.74%)
1-step:    38.7545694021 (100.0%)	 5-step:            15.14 (99.95%)
1-step:    38.7545694021 (100.0%)	 5-step:    5.12400342782 (100.0%)
1-step:    40.4603855501 (100.0%)	 5-step:    5.12400342782 (100.0%)
1-step:            35.58 (100.0%)	 5-step:    4.80039145306 (100.0%)
1-step:    38.7545694021 (100.0%)	 5-step:          20.1968 (100.0%)
1-step:           14.127 (100.0%)	 5-step:           11.639 (100.0%)
1-step:           9.6462 (100.0%)	 5-step:    4.80039145306 (100.0%)
1-step:    4.80039145306 (100.0%)	 5-step:    4.80039145306 (100.0%)
1-step:          20.1968 (100.0%)	 5-step:    4.80039145306 (100.0%)
1-step:             22.2 (78.38%)	 5-step:             22.2 (80.87%)
1-step:             15.7 (85.05%)	 5-step:             15.7 (89.07%)
1-step:    4.80039145306 (100.0%)	 5-step:             6.94 (97.3%)
1-step:              4.9 (65.19%)	 

1-step:           31.533 (91.62%)	 5-step:    4.75527763145 (99.88%)
1-step:    4.75527763145 (99.93%)	 5-step:    22.1228087893 (100.0%)
1-step:           21.302 (100.0%)	 5-step:    4.75527763145 (100.0%)
1-step:    22.1228087893 (100.0%)	 5-step:    4.75527763145 (100.0%)
1-step:    4.75527763145 (100.0%)	 5-step:             23.5 (97.1%)
1-step:    4.75527763145 (100.0%)	 5-step:      43.98106401 (100.0%)
1-step:    4.73869434202 (100.0%)	 5-step:      46.64110848 (100.0%)
1-step:    4.72708603941 (100.0%)	 5-step:          50.1635 (100.0%)
1-step:             23.5 (100.0%)	 5-step:           48.496 (100.0%)
1-step:             6.94 (100.0%)	 5-step:    39.8361197671 (100.0%)
1-step:           31.533 (100.0%)	 5-step:    38.9784604794 (100.0%)
1-step:          50.1635 (100.0%)	 5-step:    41.4676755629 (100.0%)
1-step:             46.7 (76.76%)	 5-step:             46.7 (74.57%)
1-step:    39.8361197671 (96.98%)	 5-step:        11.527577 (91.17%)
1-step:    39.6752838369 (100.0%)	 

1-step:          48.6172 (88.86%)	 5-step:    40.5655374613 (95.91%)
1-step:             49.3 (29.75%)	 5-step:    43.9934434688 (96.89%)
1-step:    45.8144862883 (99.95%)	 5-step:    37.2673841767 (99.66%)
1-step:          47.4139 (100.0%)	 5-step:    39.8198925273 (100.0%)
1-step:    43.9934434688 (88.0%)	 5-step:    39.8198925273 (40.04%)
1-step:    42.1746031748 (75.79%)	 5-step:       36.3700106 (100.0%)
1-step:             43.0 (47.43%)	 5-step:    38.7172445278 (99.33%)
1-step:    38.7172445278 (60.65%)	 5-step:    39.8198925273 (99.99%)
1-step:    44.8004053164 (66.29%)	 5-step:    43.9934434688 (68.96%)
1-step:       36.3700106 (99.03%)	 5-step:    41.5427136249 (60.1%)
1-step:    38.7172445278 (100.0%)	 5-step:    37.2471689237 (99.64%)
1-step:       36.3700106 (56.67%)	 5-step:           15.038 (99.87%)
1-step:    41.5427136249 (99.03%)	 5-step:    5.12132276142 (95.06%)
1-step:             40.1 (57.47%)	 5-step:    39.6177473384 (45.2%)
1-step:         15.76111 (100.0%)	 5-

...

1-step:     42.259612868 (53.68%)	 5-step:    47.5761395053 (31.7%)
1-step:    38.9706241313 (38.06%)	 5-step:             36.9 (56.36%)
1-step:    38.1020464961 (99.31%)	 5-step:    36.9461223283 (94.97%)
1-step:     44.108052063 (99.76%)	 5-step:    36.2742068189 (79.37%)
1-step:    46.7746962975 (65.24%)	 5-step:    36.2742068189 (92.62%)
1-step:    41.4641250513 (99.99%)	 5-step:    39.6219958359 (96.21%)
1-step:    36.9461223283 (46.43%)	 5-step:      35.20406292 (99.97%)
1-step:          34.4356 (47.18%)	 5-step:    43.2534301055 (56.75%)
1-step:    36.2742068189 (65.32%)	 5-step:           52.069 (63.95%)
1-step:    38.1020464961 (99.78%)	 5-step:    38.1020464961 (98.93%)
1-step:             43.2 (54.51%)	 5-step:             43.2 (63.38%)
1-step:             41.4 (35.84%)	 5-step:    5.53112353199 (95.82%)
1-step:           52.069 (64.71%)	 5-step:    5.53112353199 (84.91%)
1-step:    48.7488266078 (50.35%)	 5-step:    5.53112353199 (75.71%)
1-step:    44.2256364441 (93.07%)	 

1-step:    31.6440706709 (100.0%)	 5-step:    37.0295199586 (100.0%)
1-step:    31.6440706709 (100.0%)	 5-step:    48.6768332685 (99.09%)
1-step:      34.34728756 (97.48%)	 5-step:    48.6768332685 (88.88%)
1-step:             39.4 (55.32%)	 5-step:             39.4 (43.02%)
1-step:    41.4732974974 (99.88%)	 5-step:         30.05425 (73.12%)
1-step:    48.6768332685 (86.19%)	 5-step:    5.40009295809 (96.39%)
1-step:    48.6768332685 (98.81%)	 5-step:         7.105804 (99.82%)
1-step:    43.0595420789 (86.37%)	 5-step:    21.2543190776 (92.56%)
1-step:      15.03908366 (42.41%)	 5-step:    38.8847677656 (31.65%)
1-step:    5.40009295809 (96.35%)	 5-step:    5.40009295809 (97.71%)
1-step:         7.105804 (79.4%)	 5-step:    5.40009295809 (89.25%)
1-step:    21.2543190776 (92.32%)	 5-step:    5.43006507066 (87.17%)
1-step:      15.03908366 (57.7%)	 5-step:     18.428646278 (26.12%)
1-step:    5.39104554946 (85.92%)	 5-step:             21.1 (62.96%)
1-step:    5.39104554946 (52.12%)	 5

1-step:              5.0 (54.43%)	 5-step:              5.0 (50.02%)
1-step:    39.6572590117 (71.8%)	 5-step:    45.8371729454 (99.97%)
1-step:    41.2593097773 (72.45%)	 5-step:             41.6 (52.36%)
1-step:             36.1 (31.08%)	 5-step:             36.1 (60.96%)
1-step:    43.1505560532 (100.0%)	 5-step:    35.2931771863 (100.0%)
1-step:          50.9353 (57.18%)	 5-step:    38.7850239184 (78.55%)
1-step:    42.3097961347 (99.7%)	 5-step:    38.7850239184 (100.0%)
1-step:    37.9065356466 (100.0%)	 5-step:    37.9065356466 (86.11%)
1-step:    37.1706477421 (51.8%)	 5-step:    41.3615168441 (90.06%)
1-step:    37.0294534194 (99.99%)	 5-step:    41.3615168441 (99.98%)
1-step:    36.1797246415 (53.82%)	 5-step:    5.13758553796 (55.6%)
1-step:             37.1 (42.32%)	 5-step:             37.1 (55.46%)
1-step:    41.3615168441 (60.67%)	 5-step:             36.5 (59.28%)
1-step:    41.3615168441 (72.08%)	 5-step:    5.13758553796 (97.0%)
1-step:    40.5523219805 (70.42%)	 5-st

1-step:    36.1718239243 (34.04%)	 5-step:     38.621279779 (75.79%)
1-step:    43.8503508229 (99.76%)	 5-step:    35.3378597749 (97.53%)
1-step:    37.7974154461 (66.47%)	 5-step:             45.6 (41.65%)
1-step:     33.571364038 (82.21%)	 5-step:         24.61735 (53.8%)
1-step:    36.1718239243 (79.61%)	 5-step:    9.75228935598 (78.32%)
1-step:    37.1217025229 (88.75%)	 5-step:    5.20397644479 (57.37%)
1-step:             41.6 (70.49%)	 5-step:    5.20397644479 (99.55%)
1-step:             47.1 (78.37%)	 5-step:    5.20397644479 (71.62%)
1-step:    11.5012088664 (99.99%)	 5-step:    5.20397644479 (100.0%)
1-step:    5.20397644479 (100.0%)	 5-step:    15.9507216815 (100.0%)
1-step:    5.20397644479 (91.99%)	 5-step:    22.2418667733 (80.03%)
1-step:    5.17278351136 (87.97%)	 5-step:              5.3 (64.94%)
1-step:    5.21094845795 (87.97%)	 5-step:    4.84849073288 (72.8%)
1-step:              5.1 (37.05%)	 5-step:    4.84849073288 (76.07%)
1-step:    22.2418667733 (74.41%)	 5

RuntimeError: Python exception raised. Unable to extract info